Notebook containing code for optimizing hyperparameters with Optuna.

In [8]:
import sys
sys.path.append('../')

import optuna
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from src.preprocessing import preprocess_data, make_pipeline

import xgboost
from xgboost import XGBRegressor

TRAIN_SIZE = 0.8

def objective(trial):
    
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000,
        'early_stopping_rounds': 100,
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    model = XGBRegressor(**params, random_state=0)
    
    Xtrain, Xval, Ytrain, Yval = train_test_split(X_train, y_train, train_size=TRAIN_SIZE, random_state=0) 
    
    pipeline = make_pipeline(model)
    
    val_pipeline = pipeline[:-1]
    val_pipeline.fit(Xtrain, Ytrain)
    Xval_trans = val_pipeline.transform(Xval)
    
    pipeline.fit(Xtrain, Ytrain, xgb__eval_set=[(Xval_trans, Yval)], xgb__verbose=False)

    y_pred = pipeline.predict(Xval)
    r2 = r2_score(Yval, y_pred)
    
    return r2
    

X = preprocess_data()
y = X.pop('PolyPwr')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE, random_state=0) 

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_r2 = study.best_value
print("Best Hyperparameters: ", best_params)
print("Best R2: ", best_r2)


[I 2024-08-08 12:19:46,921] A new study created in memory with name: no-name-d7c4b798-e376-452e-97ab-c2cffbd36a3f


Discrete: ['Latitude', 'Longitude', 'Altitude']
Temporal: ['month', 'hour']
Continuous: ['Humidity', 'AmbientTemp', 'Wind.Speed', 'Visibility', 'Pressure', 'Cloud.Ceiling']
Categorical: ['Location', 'Season']


[I 2024-08-08 12:19:48,569] Trial 0 finished with value: 0.6621812102517947 and parameters: {'eta': 0.025547287538706612, 'max_depth': 6, 'subsample': 0.5985161440526399, 'colsample_bytree': 0.6214024416380315, 'gamma': 4.924928443849076, 'min_child_weight': 9.02128975139662, 'lambda': 3.4145968721082713, 'alpha': 9.383010582462235}. Best is trial 0 with value: 0.6621812102517947.
[I 2024-08-08 12:19:49,961] Trial 1 finished with value: 0.6652560254845852 and parameters: {'eta': 0.02377136305324193, 'max_depth': 8, 'subsample': 0.5334303515260135, 'colsample_bytree': 0.5523071601943315, 'gamma': 0.7186606504361459, 'min_child_weight': 9.067439245190764, 'lambda': 4.803223191465828, 'alpha': 0.22004896148961617}. Best is trial 1 with value: 0.6652560254845852.
[I 2024-08-08 12:19:51,899] Trial 2 finished with value: 0.6586167199080535 and parameters: {'eta': 0.029989827298090514, 'max_depth': 4, 'subsample': 0.7813650864550068, 'colsample_bytree': 0.5897703230636857, 'gamma': 3.10133719

Best Hyperparameters:  {'eta': 0.02532227240864557, 'max_depth': 9, 'subsample': 0.5701902725252715, 'colsample_bytree': 0.5700791751417876, 'gamma': 4.468310078336034, 'min_child_weight': 9.562538361029706, 'lambda': 7.292064154574558, 'alpha': 9.313263852489994}
Best R2:  0.6662327759252489


In [10]:
best_params.update({'objective': 'reg:squarederror', 'n_estimators': 100000, 'early_stopping_rounds': 100,})

In [11]:
best_params

{'eta': 0.02532227240864557,
 'max_depth': 9,
 'subsample': 0.5701902725252715,
 'colsample_bytree': 0.5700791751417876,
 'gamma': 4.468310078336034,
 'min_child_weight': 9.562538361029706,
 'lambda': 7.292064154574558,
 'alpha': 9.313263852489994,
 'objective': 'reg:squarederror',
 'n_estimators': 100000,
 'early_stopping_rounds': 100}